In [1]:
--- Libraries:
arguments = require 'Settings.arguments'
constants = require 'Settings.constants'
card_to_string = require 'Game.card_to_string_conversion'
card_tools = require 'Game.card_tools'
game_settings = require 'Settings.game_settings'
Plot = require 'itorch.Plot'
nninit = require 'nninit'
require 'torch'
require 'math'
require 'Tree.tree_builder'
require 'Tree.tree_data_generation'
require 'Tree.tree_values'
require 'Tree.tree_visualiser'
require 'nn'
require 'Tree.tree_cfr'
require 'Tree.tree_cfr2'
require 'nngraph'
require 'optim'
require 'image'
require 'NN.strategy_net_builder'
require 'NN.strategy_net_trainer'
require 'Tree.tree_warm_start'
require 'Nn.StrategyCriterion'




In [2]:
--- Create the tree
builder = PokerTreeBuilder()
--- Parameters for the tree
params = {}
params.root_node = {}
params.root_node.board = card_to_string:string_to_board('')
params.root_node.street = 1
params.root_node.current_player = constants.players.P1
params.root_node.bets = arguments.Tensor{200, 200}
--- CFR Solver
starting_ranges = arguments.Tensor(constants.players_count, game_settings.card_count)
starting_ranges[1]:copy(card_tools:get_uniform_range(params.root_node.board))
starting_ranges[2]:copy(card_tools:get_uniform_range(params.root_node.board))

In [3]:
--- Get Data from 2000 iterations
number_of_iterations = 500
train_tree = builder:build_tree(params)
data = TreeData(train_tree)
train_tree_cfr = TreeCFR()
train_tree_cfr:run_cfr(train_tree, starting_ranges,number_of_iterations)
data:get_training_set(train_tree,1)

### Training the Model and testing the model

In [10]:
-- Building the neural net model
number_iter = 500
nn_builder = StrategyNN()
nn_model = nn_builder.model
-- Building trainer:
nn_trainer = NNTrainer(data,nn_model)
-- Options:
opt = {}
opt.learningRate = 0.001
opt.momentum = 0.85
opt.validate = true
opt.weightDecay = 0.85
---opt.nesterov = true


In [11]:
strategy_criterion1 = StrategyLoss()
train_loss,test_loss = nn_trainer:train(nn_model,strategy_criterion1,opt,number_iter)

In [12]:
pred_strat = nn_trainer.model:forward({nn_trainer.all_data.features,nn_trainer.all_data.masks})
true_strat = nn_trainer.all_data.targets
abs_error =  torch.mean(torch.abs(pred_strat- true_strat))
print(abs_error)

0.25810399081988


In [ ]:
train_matrix = torch.FloatTensor()
test_matrix = torch.FloatTensor()
number_iter = 1000
abs_error_smooth = torch.FloatTensor()

for i = 1,50 do
    -- Building the neural net model
    nn_builder = StrategyNN()
    nn_model = nn_builder.model
    -- Building trainer:
    nn_trainer = NNTrainer(data,nn_model)
    -- Options:
    opt = {}
    opt.learningRate = 0.001
    opt.momentum = 0.85
    opt.validate = true
    opt.weightDecay = 0.75
    ---opt.nesterov = true
    strategy_criterion = StrategyLoss()
    ---strategy_criterion = nn.SmoothL1Criterion()
    train_loss,test_loss = nn_trainer:train(nn_model,strategy_criterion,opt,number_iter)
    train_matrix = train_matrix:cat(train_loss:view(1,number_iter),1)
    test_matrix = test_matrix:cat(test_loss:view(1,number_iter),1)
    abs_error_smooth = abs_error_smooth:cat(torch.FloatTensor(nn_trainer.table_abs_error),2)
end
print(abs_error_smooth:mean())

In [ ]:
local num_iter = torch.range(1,train_loss:size(1))
local plot = Plot()

plot:line(num_iter,train_matrix:mean(1):view(-1),'blue','Average Train Error')
plot:line(num_iter,1.96*train_matrix:std(1):view(-1)+train_matrix:mean(1):view(-1),'gray','Upper Bound 95% - Train error' )
plot:line(num_iter,-1.96*train_matrix:std(1):view(-1)+train_matrix:mean(1):view(-1),'gray','Lower Bound 95% - Train error')
plot:line(num_iter,1.96*test_matrix:std(1):view(-1)+test_matrix:mean(1):view(-1),'violet','Upper Bound 95% - Test error' )
plot:line(num_iter,-1.96*test_matrix:std(1):view(-1)+test_matrix:mean(1):view(-1),'violet','Lower Bound 95% - Test error' )
plot:line(num_iter,test_matrix:mean(1):view(-1),'red','Average Test Error')
plot:legend(true):title('Train error vs Test error - Cross Entropy Loss ')
plot:xaxis('Number of iterations'):yaxis('Loss')
plot:draw()

In [ ]:
train_matrix = torch.FloatTensor()
test_matrix = torch.FloatTensor()
number_iter = 200
abs_error_ce = torch.FloatTensor()

for i = 1,50 do
    -- Building the neural net model
    nn_builder = StrategyNN()
    nn_model = nn_builder.model
    -- Building trainer:
    nn_trainer = NNTrainer(data,nn_model)
    -- Options:
    opt = {}
    opt.learningRate = 0.001
    opt.momentum = 0.85
    opt.validate = true
    opt.weightDecay = 0.75
    ---opt.nesterov = true
    strategy_criterion = StrategyLoss()
---    strategy_criterion = nn.SmoothL1Criterion()
    train_loss,test_loss = nn_trainer:train(nn_model,strategy_criterion,opt,number_iter)
    train_matrix = train_matrix:cat(train_loss:view(1,number_iter),1)
    test_matrix = test_matrix:cat(test_loss:view(1,number_iter),1)
    abs_error_ce = abs_error_ce:cat(torch.FloatTensor(nn_trainer.table_abs_error),2)
end
print(abs_error_ce:mean())

In [ ]:
local num_iter = torch.range(1,train_loss:size(1))
local plot = Plot()

plot:line(num_iter,train_matrix:mean(1):view(-1),'blue','Average Train Error')
plot:line(num_iter,1.96*train_matrix:std(1):view(-1)+train_matrix:mean(1):view(-1),'gray','Upper Bound 95% - Train error' )
plot:line(num_iter,-1.96*train_matrix:std(1):view(-1)+train_matrix:mean(1):view(-1),'gray','Lower Bound 95% - Train error')
plot:line(num_iter,1.96*test_matrix:std(1):view(-1)+test_matrix:mean(1):view(-1),'violet','Upper Bound 95% - Test error' )
plot:line(num_iter,-1.96*test_matrix:std(1):view(-1)+test_matrix:mean(1):view(-1),'violet','Lower Bound 95% - Test error' )
plot:line(num_iter,test_matrix:mean(1):view(-1),'red','Average Test Error')
plot:legend(true):title('Train error vs Test error Huber Loss')
plot:xaxis('Number of iterations'):yaxis('Loss')
plot:draw()

In [ ]:
local num_iter = torch.range(1,train_loss:size(1))
local plot = Plot()

plot:line(num_iter,abs_error_smooth:mean(2):view(-1),'red','Average Train Error'):draw()
plot:line(num_iter,abs_error_ce:mean(2):view(-1),'blue','Average Train Error'):draw()